# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv('clean_city_data.csv')
city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mataura,-46.19,168.86,53.01,88,100,11.99,NZ,1604115127
1,Rikitea,-23.12,-134.97,76.10,64,0,2.66,PF,1604115799
2,Cabo San Lucas,22.89,-109.91,75.20,69,20,3.36,MX,1604115184
3,Ekibastuz,51.67,75.37,38.12,78,100,4.25,KZ,1604115259
4,Tocopilla,-22.09,-70.20,62.29,59,2,3.91,CL,1604115259
...,...,...,...,...,...,...,...,...,...
546,Biasca,46.36,8.97,41.00,100,0,5.12,CH,1604115810
547,San Cristobal,7.77,-72.22,67.78,92,98,2.53,VE,1604115895
548,Boshnyakovo,49.63,142.17,41.13,67,84,6.46,RU,1604115895
549,Dire Dawa,9.59,41.87,70.12,59,54,3.51,ET,1604115895


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# bring in the google API key
gmaps.configure(api_key=g_key)


In [4]:
locations = city_data[['Lat', 'Lng']]
weights = city_data['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition. For example:

# A max temperature lower than 80 degrees but higher than 70.

# Wind speed less than 10 mph.

# Zero cloudiness.

# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned 
# by your API requests to a reasonable number.

ideal_weather = city_data[(city_data['Max Temp'] >= 70) & (city_data["Max Temp"] <= 80) & (city_data['Wind Speed'] < 10) 
                          & (city_data['Cloudiness'] == 0)]
ideal_weather



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,Rikitea,-23.12,-134.97,76.10,64,0,2.66,PF,1604115799
56,Los Llanos de Aridane,28.66,-17.92,71.60,56,0,9.17,ES,1604115542
89,Agüimes,27.91,-15.45,77.00,47,0,2.24,ES,1604115817
155,Kolokani,13.57,-8.03,74.19,31,0,4.94,ML,1604115828
199,Nizwá,22.93,57.53,77.52,17,0,2.42,OM,1604115664
280,Belmonte,-15.86,-38.88,77.00,83,0,6.93,BR,1604115450
294,Ormara,25.21,64.64,78.84,25,0,6.04,PK,1604115852
299,Santo Antônio do Leverger,-15.87,-56.08,78.80,69,0,3.36,BR,1604115854
385,Caibarién,22.52,-79.47,77.00,94,0,4.70,CU,1604115868
419,Doha,25.29,51.53,79.00,69,0,4.70,QA,1604115708


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather

In [7]:
# adding a column

hotel_df['Hotel Name'] = ''
# hotel_df

<ipython-input-7-9b9fc496b560>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


In [8]:
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# using a for loop to loop over each row in a dataframe

for index, row in hotel_df.iterrows():
#   print(index, row) 
#   print(row['Lat'])

    payload = {
        'key': g_key, 
        'location': f"{row['Lat']},{row['Lng']}",
        'radius': 5000,
        'type': 'lodging',
        'keyword': 'hotel'
    }
    r = requests.get(url, params=payload)
#    pprint(r.json())
    try:
        hotel_name = r.json()['results'][0]['name']
        hotel_df.at[index, 'Hotel Name'] = hotel_name
        # print(hotel_name)
    except (IndexError, KeyError):
        print("Hotel not found")
    

hotel_df

Hotel not found
Hotel not found


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,Rikitea,-23.12,-134.97,76.10,64,0,2.66,PF,1604115799,Pension Maro'i
56,Los Llanos de Aridane,28.66,-17.92,71.60,56,0,9.17,ES,1604115542,Hotel Hacienda de Abajo
89,Agüimes,27.91,-15.45,77.00,47,0,2.24,ES,1604115817,Villa Nestor
155,Kolokani,13.57,-8.03,74.19,31,0,4.94,ML,1604115828,Maimouna sylla
199,Nizwá,22.93,57.53,77.52,17,0,2.42,OM,1604115664,Antique Inn - Nizwa
280,Belmonte,-15.86,-38.88,77.00,83,0,6.93,BR,1604115450,Pousada Belmonte
294,Ormara,25.21,64.64,78.84,25,0,6.04,PK,1604115852,Alqadri hotel zero point ormara
299,Santo Antônio do Leverger,-15.87,-56.08,78.80,69,0,3.36,BR,1604115854,
385,Caibarién,22.52,-79.47,77.00,94,0,4.70,CU,1604115868,Hotel Brisas del Mar
419,Doha,25.29,51.53,79.00,69,0,4.70,QA,1604115708,Al Najada Doha Hotel by Tivoli


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))